<a href="https://colab.research.google.com/github/blu-geek/Demo/blob/main/Preprocessing_FeaturesNew_(EDITED2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Update pip
!pip install -U pip --quiet
# Install scikit-image
!pip install -U scikit-image --quiet

import skimage
print(skimage.__version__)

     |████████████████████████████████| 2.1 MB 36.1 MB/s 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 49.2 MB/s eta 0:00:00
0.19.3


In [ ]:
!pip -qq install radiant_mlhub --quiet
!pip -qq install rasterio --quiet
!pip install --upgrade radiant-mlhub==0.3.1 --quiet
!pip install pandarallel --quiet
!pip install opencv-python --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.1/146.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 62.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
import skimage
import numpy as np
import skimage.feature
from skimage import data
from torch.utils.data import Dataset
from skimage import data, color
from skimage.feature import daisy, hog, local_binary_pattern, ORB, SIFT

In [ ]:
import os
import glob
import json
import getpass
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
from radiant_mlhub import Dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', 150)

from pandarallel import pandarallel

pandarallel.initialize(use_memory_fs=False)

os.environ['MLHUB_API_KEY'] = 'bb00e634ed60185cbc00007614254c8030fd77460421dc575efec59e0d0b1051'

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


**Downloading the competition datasets already save in gdrive using "gdown":**

In [ ]:
!gdown https://drive.google.com/uc?id=161sRO5TwcJr5V8L4e67NwoqhfI0EwawC
!unzip -q /content/ref_agrifieldnet_competition_v1.zip

Downloading...
From: https://drive.google.com/uc?id=161sRO5TwcJr5V8L4e67NwoqhfI0EwawC
To: /content/ref_agrifieldnet_competition_v1.zip
100% 342M/342M [00:09<00:00, 37.8MB/s]


# DOWNLOAD DATA FROM MLHUB

In [ ]:
# For simplicity we select 4 out 12 bands for the this baseline model

Full_bands = ['B01', 'B02', 'B03', 'B04','B05', 'B06', 'B07', 'B08','B8A', 'B09', 'B11', 'B12']

selected_bands = Full_bands
selected_bands

['B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B8A',
 'B09',
 'B11',
 'B12']

In [ ]:
# Define dataset collection_id , assets and necessary paths to collections

main = 'ref_agrifieldnet_competition_v1'

assets = ['field_ids','raster_labels']

source_collection = f'{main}_source'
train_label_collection = f'{main}_labels_train'
test_label_collection = f'{main}_labels_test'

In [ ]:
#load collection json and retrieve all unique folder ids 
#use all unique folder ids to create a list of field and label paths for all tiles

with open (f'{main}/{train_label_collection}/collection.json') as f:
    train_json = json.load(f)
    
train_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in train_json['links'][4:]]

train_field_paths = [f'{main}/{train_label_collection}/{train_label_collection}_{i}/field_ids.tif' for i in train_folder_ids]
train_label_paths = [f'{main}/{train_label_collection}/{train_label_collection}_{i}/raster_labels.tif' for i in train_folder_ids]

In [ ]:
# Create dataset for folder_ids and field_paths

competition_train_data = pd.DataFrame(train_folder_ids, columns=['unique_folder_id'])
competition_train_data['field_paths'] = train_field_paths
competition_train_data['label_paths'] = train_label_paths
competition_train_data.head()

,unique_folder_id,field_paths,label_paths
0,28852,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_28852/field_ids.tif,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_28852/raster_labels.tif
1,d987c,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_d987c/field_ids.tif,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_d987c/raster_labels.tif
2,ca1d4,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_ca1d4/field_ids.tif,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_ca1d4/raster_labels.tif
3,2ec18,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_2ec18/field_ids.tif,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_2ec18/raster_labels.tif
4,7575d,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_7575d/field_ids.tif,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_7575d/raster_labels.tif


In [ ]:
with open (f'{main}/{test_label_collection}/collection.json') as f:
    test_json = json.load(f)
    
test_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in test_json['links'][4:]]

test_field_paths = [f'{main}/{test_label_collection}/{test_label_collection}_{i}/field_ids.tif' for i in test_folder_ids]

In [ ]:
competition_test_data = pd.DataFrame(test_folder_ids , columns=['unique_folder_id'])
competition_test_data['field_paths'] = test_field_paths
competition_test_data.head()

,unique_folder_id,field_paths
0,6199c,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_test/ref_agrifieldnet_competition_v1_labels_test_6199c/field_ids.tif
1,6c81d,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_test/ref_agrifieldnet_competition_v1_labels_test_6c81d/field_ids.tif
2,1ebeb,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_test/ref_agrifieldnet_competition_v1_labels_test_1ebeb/field_ids.tif
3,586a2,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_test/ref_agrifieldnet_competition_v1_labels_test_586a2/field_ids.tif
4,65812,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_test/ref_agrifieldnet_competition_v1_labels_test_65812/field_ids.tif


# PREPROCESS FIELDS AND CROPS IN TILES FOR TRAININIG

In [ ]:
from skimage.feature import daisy, hog, local_binary_pattern, ORB, SIFT

In [ ]:
#Extract field_crop Pairs dataframe 
def load_image(tile_id, path=source_collection, main=main, selected_bands=selected_bands):
    bands_src = [rasterio.open(f'{main}/{path}/{path}_{tile_id}/{band}.tif') for band in selected_bands]
    bands_array = np.stack([band.read(1) for band in bands_src], axis=0)
    return bands_array

def get_field_crop(df, path=source_collection, main=main, selected_bands=selected_bands):
    field_crops = []

    for idx in tqdm(df.index):
        row = df.loc[idx]
        label_field_file = row['unique_folder_id']
        field_path = row['field_paths']
        with rasterio.open(field_path) as src:
            field_data = src.read()[0]
        crop_data = None
        if 'label_paths' in row.keys():
            label_path = row['label_paths']
            with rasterio.open(label_path) as src:
                crop_data = src.read()[0]
        
        for field_id in np.unique(field_data):
            if field_id == 0:
                continue
            mask = field_data == field_id
            y, x = np.where(mask)
            top = y.min()
            left = x.min()
            height = y.max() - y.min() + 1
            width = x.max() - x.min() + 1
            
            field_crop = dict(
                file=label_field_file, 
                field_path=field_path,
                field_id=field_id,
                top=top,
                left=left,
                height=height,
                width=width,
                area=height*width,
                bbox=[top, left, height, width]
            )
            if crop_data is not None:
                field_crop.update({'label_path': label_path})
            
            fm = mask[top:top+height, left:left+width]
            if crop_data is not None:
                crop = (crop_data[top:top+height, left:left+width] * fm).max()
                field_crop['crop'] = crop
            field_crops.append(field_crop)

    field_crops = pd.DataFrame(field_crops)

    return field_crops

def get_field_features(df, path=source_collection, main=main, selected_bands=selected_bands):
    field_features = []

    for idx in tqdm(df.index):
        row = df.loc[idx]
        label_field_file = row['unique_folder_id']
        field_path = row['field_paths']
        with rasterio.open(field_path) as src:
            field_data = src.read()[0]
        image = load_image(tile_id=label_field_file, path=path, 
                       main=main, selected_bands=selected_bands)
        
        field_id = field_data.flatten()[(field_data != 0).flatten()]
        field_feature = dict(field_id=field_id)
        features  = (image * (field_data != 0)).reshape(12, -1).T[(field_data != 0).flatten()]
        features = {key: features[:, i] for i, key in enumerate(band_mapping.keys())}
        field_feature.update(features)
        field_features.append(field_feature)
    field_features = pd.DataFrame(field_features)

    return field_features

In [ ]:
def get_features(df, path=source_collection, main=main, selected_bands=selected_bands):
    label_field_file = df['file']
    field_path = df['field_path']
    with rasterio.open(field_path) as src:
        field_data = src.read()[0]
    crop_data = None
    if 'label_path' in df.keys():
        label_path = df['label_path']
        with rasterio.open(label_path) as src:
            crop_data = src.read()[0]
    top = df['top']
    left = df['left']
    height = df['height']
    width = df['width']
    field_id = df['field_id']
    
    mask = field_data == field_id
    fm = mask[top:top+height, left:left+width]
    
    image = load_image(tile_id=label_field_file, path=path, 
                       main=main, selected_bands=selected_bands)
    image = image[:, top:top+height, left:left+width]  # * fm
    features = np.array([])
    for im in image:
        band = (im * fm).flatten()
        band = pd.Series(band[band > 0])
        f_band = band.agg(['mean', 'std', 'mad', 'kurtosis', 'skew', 'min', 'max']).fillna(0).values.T.flatten()
        f_hog = hog(im, orientations=9, pixels_per_cell=(3, 3), cells_per_block=(1, 1), feature_vector=False)
        f_hog = pd.DataFrame(f_hog.reshape(-1, 9))
        f_hog = f_hog.agg(['mean', 'std', 'mad', 'kurtosis', 'skew', 'min', 'max']).fillna(0).values.T.flatten()

        h_band = np.histogram(band, bins=10, density=True)[0]
        features = np.concatenate([features, f_band, f_hog, h_band])
    f_hog1 = hog(image * fm, channel_axis=0, orientations=9, pixels_per_cell=(3, 3), cells_per_block=(1, 1), feature_vector=False)
    f_hog1 = pd.DataFrame(f_hog1.reshape(-1, 9))
    f_hog1 = f_hog1.agg(['mean', 'std', 'mad', 'kurtosis', 'skew', 'min', 'max']).fillna(0).values.T.flatten()   
    features = np.concatenate([features, f_hog1])

    return pd.Series(features)

In [ ]:
# https://plos.figshare.com/articles/dataset/_Formulas_of_several_commonly_used_vegetation_indices_/1103896

# some important constant
# L, X, Y, and Z are soil background adjusted factors
L = 0.5
# X =
Y = 0.16
# Z =

# a and b are the slope and intercept of the soil line
a = 0.3
b = 0.5


# Simple ratio (SR)
def sr(NIR, R):
    return NIR/R


# Normalized difference vegetation index (NDVI)
def ndvi(NIR, R):
    return (NIR-R)/(NIR+R)

# Perpendicular vegetation index (PVI)
def pvi(NIR, R):
    a = 0.3
    b = 0.5
    
    PVI = (NIR - a * R - b)/(a**2 + 1)**(1/2)
    return PVI


# Soil adjusted vegetation index (SAVI)
def savi(NIR, R):
    L = 0.5
    
    SAVI = (NIR-R) / (NIR+R+L) * (1 + L)
    return SAVI

# Weighted difference vegetation index (WDVI)
def wdvi(NIR, R):
    WDVI = NIR - a*R
    return WDVI


# Transformed soil adjusted vegetation index (TSAVI)
def tsavi(NIR, R):
    a = 0.3
    b = 0.5
#     X = ?
    TSAVI = a*(NIR-a*R-b) / (R+a*(NIR-b)+ X*(1+a**2))
    pass  # TODO


# Modified soil adjusted vegetation index (MSAVI)
def msavi(NIR, R):
    return (2 * NIR + 1 - ((2*NIR+1)**2 - 8*(NIR - R))**(1/2) ) / 2


# Optimized soil adjusted vegetation index (OSAVI)
def osavi(NIR, R):
    Y = 0.16
    OSAVI = 1.6 * (NIR-R) / (NIR + R + Y)
    return OSAVI

# Generalized soil adjusted vegetation index (GESAVI)
def gesavi(NIR, R):
#     Z = ?
    GESAVI = (NIR - a*R - b) / (R + Z)
    pass  # TODO


#Atmospherically resistant vegetation index (ARVI)
def arvi(NIR, R, B):
    gamma = -1
    RB = R - gamma * (B - R)
    ARVI = ndvi(NIR, RB)   # (NIR-RB) / (NIR+RB)
    return ARVI


# Modified normalized difference vegetation index (MNDVI)
def mndvi(NIR, R, SWIR):
    NDVI = ndvi(NIR, R)
    SWIRmax = SWIR.max()
    SWIRmin = SWIR.min()
    MNDVI = NDVI * (SWIRmax - SWIR) / (SWIRmax - SWIRmin)
    return MNDVI


# Enhanced vegetation index (EVI)
def evi(NIR, R, B):
    with np.errstate(divide='ignore', invalid='ignore'):
        EVI = 2.5*((NIR-R)/(NIR+6*R-7.5*B+1))
    return EVI


# Reduced simple ratio (RSR)
def rsr(NIR, R, SWIR):
    SR = sr(NIR, R)
    SWIRmax = SWIR.max()
    SWIRmin = SWIR.min()
    RSR = SR * (SWIRmax - SWIR) / (SWIRmax - SWIRmin)
    return RSR


# Moisture adjusted vegetation index (MAVI)
def mavi(NIR, R, SWIR):
    MAVI = (NIR - R) / (NIR+R+SWIR)
    return MAVI



# Modefied Simple Ratio (MSR)
def msr(NIR, R):
    MSR =  (NIR/R - 1) / (NIR/R + 1)**(1/2)
    return MSR


############ 
def s2rep(R, R_EDGE1, R_EDGE2, R_EDGE3):
    S2REP = 705 + 35 * ( ( ((R_EDGE3 + R)/2) - R_EDGE1)/ (R_EDGE2 - R_EDGE1) ) 
    return S2REP

def ccci(NIR, R, R_EDGE1):
    NDVI_R = ndvi(NIR, R)
    NDVI_R_EDGE1 = ndvi(NIR, R_EDGE1)
    with np.errstate(divide='ignore', invalid='ignore'):
        CCCI = NDVI_R_EDGE1 / NDVI_R
    return CCCI


def mcari(G, R, R_EDGE1):
    MCARI = ((R_EDGE1 - R) - 2 * (R_EDGE1 - G)) * (R_EDGE1 / R)
    return MCARI


def tcari(G, R, R_EDGE1):
    TCARI = 3 * ((R_EDGE1 - R) - 0.2 * (R_EDGE1 - G) * (R_EDGE1 / R))
    return TCARI


def sipi(NIR, R, B):
    with np.errstate(divide='ignore', invalid='ignore'):
        SIPI = (NIR - B) / (NIR - R)
    return SIPI


def mresr(R_EDGE1, R_EDGE2, C):
    with np.errstate(divide='ignore', invalid='ignore'):
        MRESR = (R_EDGE2 - C) / (R_EDGE1 - C)
    return MRESR


# Difference Vegetation Index (DVI)
def dvi(NIR, R):
#     DVI = b08 - b04
    DVI = NIR - R
    return DVI


# Ratio Vegetation Index (RVI)
def rvi(NIR, R):
    RVI = SR(NIR, R)
    return RVI

# Simple Ratio Index (SRI)  {inverse of SR}
def sri(NIR, R):
    SRI = SR(R, NIR)
    return SRI

# Return Difference Vegetation Index (RDVI)
def rdvi1(NIR, R):
    DVI = dvi(NIR, R)
    NDVI = ndvi(NIR, R)
    
    RDVI = (NDVI*DVI)**(1/2.0)
    return RDVI


# Green Normalized Difference Vegetation Index (GNDVI)
def gndvi(NIR, G):
    return NDVI(NIR, G)

# Normalized Difference Red-edge Index (RDVI)
def ndre(NIR, R_EDGE1):
    return NDVI(NIR, R_EDGE1)


# Renormalized Difference Vegetation Index (RDVI)
def rdvi2(NIR, R): 
    RDVI = (NIR-R)/((NIR+R)**(1/2.0)) 
    return RDVI


# Modefied Green Red Vegetation Index
def mgvri(G, R):
    MGVRI = (((G)**(2)) - ((R)**(2))) / (((G)**(2)) + (R**(2)))
    return MGVRI


# Green Leaf Index
def gli(B, G, R):
    with np.errstate(divide='ignore', invalid='ignore'):
        GLI = (2*G-(R+B)) / (2*G+(R+B))
    return GLI


# Modefied Photochemical Reflectance 
def mpri(G, R):
    MPRI = NDVI(G, R)
    return MPRI


# Red Green Blue Vegetation Index
def rgvbi(B, G, R):
    with np.errstate(divide='ignore'):
        RGVBI = (G - (B*R)) / (G**2 + B*R)
    return RGVBI


# Excess of Green
def exg(B, G, R):
#     EXG = 2 * b03 - b04 - b02
    EXG = 2 * G - R - B
    return EXG


# Vegetativen
def veg(B, G, R):
    alpha = 0.667
    VEG = G / (R**alpha + B**(1-alpha))
    return VEG


def ci_green(NIR, G):
    CI_green = sr(NIR, G) - 1
    return CI_green

# unnamed
def oass(NIR, R, R_EDGE1):
    with np.errstate(divide='ignore', invalid='ignore'):
        OASS1 = ((NIR - R_EDGE1)/(NIR - R))
    return OASS1

# unnamed
def mtci(NIR, R, R_EDGE1):
    with np.errstate(divide='ignore', invalid='ignore'):
        MTCI = ((NIR - R_EDGE1)/(R_EDGE1 - R))
    return MTCI


def ndi(G, R):
#     NDI = 128 * (((b03-b04) / (b03+b04)) + 1)
    NDI = 128 * (((G-R) / (G+R)) + 1)
    return NDI

def exr(G, R):
    ExR = 1.3*R - G
    return ExR

def cive(B, G, R):
    CIVE = 0.441*R - 0.811*G + 0.385*R + 18.787
    return CIVE


def exgr(B, G, R):
    ExG = exg(B, G, R)
    ExR = exr(G, R)
    ExGR = ExG - ExR
    return ExGR


def comi(B, G, R):
    ExG = exg(B, G, R)
    ExR = exr(G, R)
    VEG = veg(B, G, R)
    CIVE = cive(B, G, R)
    COMI = 2 * ExG + CIVE - ExR + VEG
    return COMI


def mexg(B, G, R):
    MExG = 1.262*G - 0.884*R - 0.331*B
    return MExG


def com2(B, G, R):
    ExG = exg(B, G, R)
    CIVE = cive(B, G, R)
    VEG = veg(B, G, R)
    
    COM2 = 0.36*ExG + 0.47*CIVE + 0.17*VEG
    return COM2


def gb(B, G):
    GB = DVI(G, B)
    return GB

def tdvi(NIR, R):
    NDVI = ndvi(NIR, R)
    with np.errstate(divide='ignore', invalid='ignore'):
        TDVI = (0.5 + NDVI)**(1/2.0)
    return TDVI

def ndvi_gb(NIR, B, G):
    GB = G + B
    NDVI_GB = ndvi(NIR, GB)
    return NDVI_GB

# hsi format
def intensity(R, G, B):
    return (R+G+B)/3

# hsi format
def saturation(R, G, B=0):
    rgb = np.concatenate([R, G, B], axis=0)
    min_value = np.min(rgb, axis=0)
    return 1 - min_value / intensity(R, G, B)



In [ ]:
##### NEW ONES
# transformed vegetation index
def tvi(NIR, R):
    RVI = sr(NIR, R)
    with np.errstate(divide='ignore', invalid='ignore'):
        TVI = (0.5 + RVI)**(1/2.0)
    return TVI

# square root vegetation index
def srrvi(NIR, R):
    RVI = sr(NIR, R)
    with np.errstate(divide='ignore', invalid='ignore'):
        SRRVI = (RVI)**(1/2.0)
    return SRRVI

def transform_index(index):
    with np.errstate(divide='ignore', invalid='ignore'):
        TVI = (0.5 + index)**(1/2.0)
    return TVI

def square_root_index(index):
    with np.errstate(divide='ignore', invalid='ignore'):
        SRRVI = (RVI)**(1/2.0)
    return SRRVI

# Transform 
def brightness(R, R_EDGE1, R_EDGE2, R_EDGE3):
    return 32 + 0.433*R + 0.632*R_EDGE1 + 0.586*R_EDGE2 + 0.264*R_EDGE3

def greenness(R, R_EDGE1, R_EDGE2, R_EDGE3):
    return 32 - 0.290*R - 0.562*R_EDGE1 + 0.600*R_EDGE2 + 0.491*R_EDGE3

def yellowness(R, R_EDGE1, R_EDGE2, R_EDGE3):
    return 32 - 0.829*R - 0.522*R_EDGE1 - 0.039*R_EDGE2 + 0.194*R_EDGE3

def nonesuch(R, R_EDGE1, R_EDGE2, R_EDGE3):
    return 32 + 0.223*R + 0.012*R_EDGE1 - 0.543*R_EDGE2 + 0.810*R_EDGE3

# normalize the NDVI to +-1
def slope(NDVI):
    return (4/np.pi) * np.atan(NDVI)

def slope1(NIR, R):
    return (4/np.pi) * np.arctan((NIR-R)/(NIR+R))

def slope2(NIR, R, B):
    DENOM = np.sqrt(B**2 + R**2 + NIR**2)
    NUM = -0.4167*B - 0.3317*R + 0.8464*NIR
    return (4/np.pi) * np.arctan(NUM/DENOM)

def slope3(NIR, R, B):
    DENOM = np.sqrt(B**2 + R**2 + NIR**2)
    NUM = 0.6864*B + 0.7253*R + 0.0537*NIR
    return (4/np.pi) * np.arctan(NUM/DENOM)

In [ ]:
band_mapping = {
    "B01": "coastal",     # C
    "B02": "blue",        # B
    "B03": "green",       # G
    "B04": "red",         # R
    "B05": "rededge",     # R_EDGE1
    "B06": "rededge",     # R_EDGE2
    "B07": "rededge",     # R_EDGE3
    "B08": "nir",         # NIR
    "B8A": "nir08",       # NIR_8
    "B09": "nir09",       # NIR_9
    "B11": "swir16",      # SWIR
    "B12": "swir22"       # SWIR_12
}

In [ ]:
from itertools import combinations, permutations

In [ ]:
band_pos = {key: index for index, key in enumerate(band_mapping.keys())}
band_pos

{'B01': 0,
 'B02': 1,
 'B03': 2,
 'B04': 3,
 'B05': 4,
 'B06': 5,
 'B07': 6,
 'B08': 7,
 'B8A': 8,
 'B09': 9,
 'B11': 10,
 'B12': 11}

In [ ]:
train_field_crop = get_field_crop(competition_train_data)

100%|██████████| 1165/1165 [00:07<00:00, 163.01it/s]


In [ ]:
train_field_crop.head(2)

,file,field_path,field_id,top,left,height,width,area,bbox,label_path,crop
0,28852,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_28852/field_ids.tif,756,45,40,5,4,20,"[45, 40, 5, 4]",ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_28852/raster_labels.tif,6
1,28852,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_28852/field_ids.tif,757,43,23,6,8,48,"[43, 23, 6, 8]",ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_28852/raster_labels.tif,6


In [ ]:
%%time
train_field_features = get_field_features(competition_train_data)
train_field_features = train_field_features.explode(column=train_field_features.columns.tolist(),
                                                   ignore_index=True)


100%|██████████| 1165/1165 [00:28<00:00, 41.01it/s]


CPU times: user 27 s, sys: 1.87 s, total: 28.9 s
Wall time: 28.7 s


In [ ]:
train_field_features.head(2)

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12
0,757,43,39,38,38,41,54,63,61,64,12,57,37
1,757,43,39,38,38,42,57,67,63,72,12,63,42


In [ ]:
train_field_features.nunique()

field_id    5551
B01           23
B02           31
B03           41
B04           63
B05           57
B06           65
B07           78
B08           80
B8A           86
B09           22
B11          101
B12           84
dtype: int64

In [ ]:
train_field_features.astype('float').describe()

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12
count,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000,188228.000000
mean,4028.345108,43.461122,39.452738,38.907378,42.401221,44.988280,55.510604,63.687937,60.426897,70.171919,13.541662,73.128461,52.639634
std,2086.157338,2.920023,3.565797,4.802049,8.261763,7.357721,6.575534,7.818992,7.572427,8.871060,2.622812,15.530609,13.903433
min,1.000000,29.000000,22.000000,19.000000,16.000000,23.000000,27.000000,30.000000,27.000000,29.000000,7.000000,14.000000,7.000000
25%,2174.000000,42.000000,38.000000,36.000000,37.000000,41.000000,51.000000,59.000000,56.000000,65.000000,12.000000,67.000000,47.000000
50%,4468.000000,44.000000,40.000000,39.000000,43.000000,45.000000,56.000000,64.000000,61.000000,71.000000,13.000000,76.000000,55.000000
75%,5841.000000,45.000000,42.000000,42.000000,48.000000,50.000000,60.000000,69.000000,65.000000,76.000000,15.000000,83.000000,62.000000
max,7332.000000,51.000000,52.000000,59.000000,78.000000,79.000000,94.000000,113.000000,106.000000,121.000000,28.000000,114.000000,90.000000


In [ ]:
test_field_crop = get_field_crop(competition_test_data)  #, include_bands=True)

100%|██████████| 707/707 [00:02<00:00, 298.44it/s]


In [ ]:
test_field_crop.head(2)

,file,field_path,field_id,top,left,height,width,area,bbox
0,6199c,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_test/ref_agrifieldnet_competition_v1_labels_test_6199c/field_ids.tif,5407,137,205,18,12,216,"[137, 205, 18, 12]"
1,6c81d,ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_test/ref_agrifieldnet_competition_v1_labels_test_6c81d/field_ids.tif,6309,227,245,9,3,27,"[227, 245, 9, 3]"


In [ ]:
%%time
test_field_features = get_field_features(competition_test_data)
test_field_features = test_field_features.explode(column=test_field_features.columns.tolist(),
                                                   ignore_index=True)


100%|██████████| 707/707 [00:17<00:00, 41.09it/s]

CPU times: user 16.3 s, sys: 1.17 s, total: 17.4 s
Wall time: 17.3 s


In [ ]:
test_field_features.head(2)

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12
0,5407,39,35,35,35,38,48,55,59,60,11,53,39
1,5407,39,34,33,34,37,49,58,58,63,11,54,40


In [ ]:
train_field_features.head(2)

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12
0,757,43,39,38,38,41,54,63,61,64,12,57,37
1,757,43,39,38,38,42,57,67,63,72,12,63,42


In [ ]:
train_field_features['Train'] = 1
test_field_features['Train'] = 0

field_features = pd.concat([train_field_features, test_field_features], axis=0)
field_features[list(band_mapping.keys())] = field_features[list(band_mapping.keys())].astype('float32')

field_features[list(band_mapping.keys())] = field_features[list(band_mapping.keys())] ** (0.5)  # reduce noise

In [ ]:
field_features.head(2)

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,Train
0,757,6.557439,6.244998,6.164414,6.164414,6.403124,7.348469,7.937254,7.810250,8.000000,3.464102,7.549834,6.082763,1
1,757,6.557439,6.244998,6.164414,6.164414,6.480741,7.549834,8.185353,7.937254,8.485281,3.464102,7.937254,6.480741,1


In [ ]:
band_rename = {
    "B01": "coastal",     # C
    "B02": "blue",        # B
    "B03": "green",       # G
    "B04": "red",         # R
    "B05": "rededge1",    # R_EDGE1
    "B06": "rededge2",    # R_EDGE2
    "B07": "rededge3",    # R_EDGE3
    "B08": "nir",         # NIR
    "B8A": "nir08",       # NIR_8
    "B09": "nir09",       # NIR_9
    "B11": "swir16",      # SWIR
    "B12": "swir22"       # SWIR_12
}

In [ ]:
field_features.rename(columns=band_rename, inplace=True)
field_features.head(2)

,field_id,coastal,blue,green,red,rededge1,rededge2,rededge3,nir,nir08,nir09,swir16,swir22,Train
0,757,6.557439,6.244998,6.164414,6.164414,6.403124,7.348469,7.937254,7.810250,8.000000,3.464102,7.549834,6.082763,1
1,757,6.557439,6.244998,6.164414,6.164414,6.480741,7.549834,8.185353,7.937254,8.485281,3.464102,7.937254,6.480741,1


In [ ]:
%%time
# Get intensities and saturations
order = {
    'rgb': ['red', 'green', 'blue'],
    'rg': ['red', 'green'],
    'rb': ['red', 'blue'],
    'gb': ['green', 'blue'],
    'nir': ['nir', 'nir08', 'nir09'],
    'swir': ['swir16', 'swir22'],
    'rededge': ['rededge1', 'rededge2', 'rededge3'],
    'rededge12': ['rededge1', 'rededge2'],
    'rededge13': ['rededge1', 'rededge3'],
    'rededge23': ['rededge2', 'rededge3'],
}
for key, value in order.items():
    field_features[f'intensity_{key}'] = field_features[value].mean(axis=1)
    field_features[f'saturation_{key}'] = 1 - (field_features[value].min(axis=1) / field_features[f'intensity_{key}'])

CPU times: user 510 ms, sys: 79 ms, total: 589 ms
Wall time: 586 ms


In [ ]:
field_features.head(2)

,field_id,coastal,blue,green,red,rededge1,rededge2,rededge3,nir,nir08,...,intensity_swir,saturation_swir,intensity_rededge,saturation_rededge,intensity_rededge12,saturation_rededge12,intensity_rededge13,saturation_rededge13,intensity_rededge23,saturation_rededge23
0,757,6.557439,6.244998,6.164414,6.164414,6.403124,7.348469,7.937254,7.810250,8.000000,...,6.816298,0.107615,7.229616,0.114320,6.875797,0.068744,7.170189,0.106980,7.642862,0.038519
1,757,6.557439,6.244998,6.164414,6.164414,6.480741,7.549834,8.185353,7.937254,8.485281,...,7.208997,0.101021,7.405309,0.124852,7.015288,0.076197,7.333047,0.116228,7.867594,0.040388


In [ ]:
%%time
# get ndvi and some other 2 input vegetation indices
order = [
    ['nir', 'red'],
    ['nir', 'green'],
    ['nir', 'blue'],
    ['nir', 'coastal'],
    ['nir', 'intensity_rgb'],
    ['nir', 'intensity_rededge'],
    ['nir08', 'red'],
    ['nir08', 'green'],
    ['nir08', 'blue'],
    ['nir08', 'coastal'],
    ['nir08', 'intensity_rgb'],
    ['nir08', 'intensity_rededge'],
    ['nir09', 'red'],
    ['nir09', 'green'],
    ['nir09', 'blue'],
    ['nir09', 'coastal'],
    ['nir09', 'intensity_rgb'],
    ['nir09', 'intensity_rededge'],
    ['intensity_nir', 'red'],
    ['intensity_nir', 'green'],
    ['intensity_nir', 'blue'],
    ['intensity_nir', 'coastal'],
    ['intensity_nir', 'intensity_rgb'],
    ['intensity_nir', 'intensity_rededge'],
    ['saturation_nir', 'saturation_rgb'],

    # swir
    ['swir16', 'red'],
    ['swir16', 'green'],
    ['swir16', 'blue'],
    ['swir16', 'coastal'],
    ['swir16', 'intensity_rededge'],
    ['swir16', 'intensity_rgb'],
    ['swir22', 'red'],
    ['swir22', 'green'],
    ['swir22', 'blue'],
    ['swir22', 'coastal'],
    ['swir22', 'intensity_rededge'],
    ['swir22', 'intensity_rgb'],
    ['intensity_swir', 'red'],
    ['intensity_swir', 'green'],
    ['intensity_swir', 'blue'],
    ['intensity_swir', 'coastal'],
    ['intensity_swir', 'intensity_rededge'],
    ['intensity_swir', 'intensity_rgb'],
    ['saturation_swir', 'saturation_rgb'],

    # coastal
    ['coastal', 'red'],
    ['coastal', 'green'],
    ['coastal', 'blue'],
    ['coastal', 'intensity_rgb'],
    ['coastal', 'intensity_rededge'],   
]

band_1, band_2 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)

funcs = [sr, ndvi, pvi, savi, wdvi, msavi, osavi, msr,
         dvi, exr, tdvi, rdvi1, rdvi2, slope1, tvi, srrvi]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}' for a, b in order]] = func(field_features[band_1].values,
                                                               field_features[band_2].values)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in true_divide


CPU times: user 7.3 s, sys: 1.33 s, total: 8.63 s
Wall time: 8.57 s


In [ ]:
field_features.head(2)

,field_id,coastal,blue,green,red,rededge1,rededge2,rededge3,nir,nir08,...,srrvi_intensity_swir_blue,srrvi_intensity_swir_coastal,srrvi_intensity_swir_intensity_rededge,srrvi_intensity_swir_intensity_rgb,srrvi_saturation_swir_saturation_rgb,srrvi_coastal_red,srrvi_coastal_green,srrvi_coastal_blue,srrvi_coastal_intensity_rgb,srrvi_coastal_intensity_rededge
0,757,6.557439,6.244998,6.164414,6.164414,6.403124,7.348469,7.937254,7.810250,8.000000,...,1.044740,1.019547,0.970994,1.049263,4.980382,1.031386,1.031386,1.02471,1.029146,0.952378
1,757,6.557439,6.244998,6.164414,6.164414,6.480741,7.549834,8.185353,7.937254,8.485281,...,1.074413,1.048505,0.986656,1.079064,4.825375,1.031386,1.031386,1.02471,1.029146,0.941013


In [ ]:
%%time
# get some other 2 input vegetation indices (category based)
order = [
    # rgb
    ['green', 'red'],
    ['green', 'blue'],
    ['red', 'blue'],
    ['red', 'green'],
    ['blue', 'red'],
    ['blue', 'green'],
    ['intensity_rg', 'intensity_rb'],
    ['intensity_rg', 'intensity_gb'],
    ['intensity_rb', 'intensity_rg'],
    ['intensity_rb', 'intensity_gb'],
    ['intensity_gb', 'intensity_rg'],
    ['intensity_gb', 'intensity_rb'],
    # swir
    ['swir16', 'swir22'],
    ['swir22', 'swir16'],
    # rededge
    ['rededge1', 'rededge2'],
    ['rededge1', 'rededge3'],
    ['rededge2', 'rededge3'],
    ['rededge2', 'rededge1'],
    ['rededge3', 'rededge1'],
    ['rededge3', 'rededge2'],
    ['intensity_rededge12', 'intensity_rededge13'],
    ['intensity_rededge12', 'intensity_rededge23'],
    ['intensity_rededge23', 'intensity_rededge13'],
    ['intensity_rededge23', 'intensity_rededge12'],
    ['intensity_rededge13', 'intensity_rededge12'],
    ['intensity_rededge13', 'intensity_rededge23'],
    
]

band_1, band_2 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)

funcs = [mgvri, ndvi, ndi, exr, dvi]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}' for a, b in order]] = func(field_features[band_1].values,
                                                                          field_features[band_2].values)

CPU times: user 3.59 s, sys: 2.33 s, total: 5.92 s
Wall time: 5.92 s


In [ ]:
field_features.head(2)

,field_id,coastal,blue,green,red,rededge1,rededge2,rededge3,nir,nir08,...,dvi_rededge2_rededge3,dvi_rededge2_rededge1,dvi_rededge3_rededge1,dvi_rededge3_rededge2,dvi_intensity_rededge12_intensity_rededge13,dvi_intensity_rededge12_intensity_rededge23,dvi_intensity_rededge23_intensity_rededge13,dvi_intensity_rededge23_intensity_rededge12,dvi_intensity_rededge13_intensity_rededge12,dvi_intensity_rededge13_intensity_rededge23
0,757,6.557439,6.244998,6.164414,6.164414,6.403124,7.348469,7.937254,7.810250,8.000000,...,-0.588785,0.945345,1.534130,0.588785,-0.294392,-0.767065,0.472672,0.767065,0.294392,-0.472672
1,757,6.557439,6.244998,6.164414,6.164414,6.480741,7.549834,8.185353,7.937254,8.485281,...,-0.635518,1.069094,1.704612,0.635518,-0.317759,-0.852306,0.534547,0.852306,0.317759,-0.534547


In [ ]:
%%time
# get some other 3 input vegetation indices 
order = [
    # nir & rgb
    ['nir', 'red', 'blue'],
    ['nir', 'blue', 'red'],
    ['nir', 'red', 'green'],
    ['nir', 'green', 'red'],
    ['nir', 'blue', 'green'],
    ['nir', 'green', 'blue'],
    ['nir', 'blue', 'green'],
    # nir08 & rgb
    ['nir08', 'red', 'blue'],
    ['nir08', 'blue', 'red'],
    ['nir08', 'red', 'green'],
    ['nir08', 'green', 'red'],
    ['nir08', 'blue', 'green'],
    ['nir08', 'green', 'blue'],
    ['nir08', 'blue', 'green'],
    # nir09 & rgb
    ['nir09', 'red', 'blue'],
    ['nir09', 'blue', 'red'],
    ['nir09', 'red', 'green'],
    ['nir09', 'green', 'red'],
    ['nir09', 'blue', 'green'],
    ['nir09', 'green', 'blue'],
    ['nir09', 'blue', 'green'],
    # coastal & rgb
    ['coastal', 'red', 'blue'],
    ['coastal', 'blue', 'red'],
    ['coastal', 'red', 'green'],
    ['coastal', 'green', 'red'],
    ['coastal', 'blue', 'green'],
    ['coastal', 'green', 'blue'],
    ['coastal', 'blue', 'green'],
    # intensity_nir
    ['intensity_nir', 'red', 'blue'],
    ['intensity_nir', 'blue', 'red'],
    ['intensity_nir', 'red', 'green'],
    ['intensity_nir', 'green', 'red'],
    ['intensity_nir', 'blue', 'green'],
    ['intensity_nir', 'green', 'blue'],
    ['intensity_nir', 'blue', 'green'],
     # intensity_swir
    ['intensity_swir', 'red', 'blue'],
    ['intensity_swir', 'blue', 'red'],
    ['intensity_swir', 'red', 'green'],
    ['intensity_swir', 'green', 'red'],
    ['intensity_swir', 'blue', 'green'],
    ['intensity_swir', 'green', 'blue'],
    ['intensity_swir', 'blue', 'green'],
    # intensity_rededge
    ['intensity_rededge', 'red', 'blue'],
    ['intensity_rededge', 'blue', 'red'],
    ['intensity_rededge', 'red', 'green'],
    ['intensity_rededge', 'green', 'red'],
    ['intensity_rededge', 'blue', 'green'],
    ['intensity_rededge', 'green', 'blue'],
    ['intensity_rededge', 'blue', 'green'], 
]

band_1, band_2, band_3 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)
band_3 = list(band_3)

funcs = [arvi, evi, sipi, slope2, slope3]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}_{c}' for a, b, c in order]] = func(field_features[band_1].values,
                                                                                 field_features[band_2].values,
                                                                                 field_features[band_3].values
                                                                                )

CPU times: user 5.64 s, sys: 8.49 s, total: 14.1 s
Wall time: 14.1 s


In [ ]:
field_features.head(2)

,field_id,coastal,blue,green,red,rededge1,rededge2,rededge3,nir,nir08,...,slope3_intensity_swir_red_green,slope3_intensity_swir_green_red,slope3_intensity_swir_blue_green,slope3_intensity_swir_green_blue,slope3_intensity_rededge_red_blue,slope3_intensity_rededge_blue_red,slope3_intensity_rededge_red_green,slope3_intensity_rededge_green_red,slope3_intensity_rededge_blue_green,slope3_intensity_rededge_green_blue
0,757,6.557439,6.244998,6.164414,6.164414,6.403124,7.348469,7.937254,7.810250,8.000000,...,0.874071,0.874071,0.875544,0.875329,0.862526,0.862739,0.861169,0.861169,0.862739,0.862526
1,757,6.557439,6.244998,6.164414,6.164414,6.480741,7.549834,8.185353,7.937254,8.485281,...,0.861815,0.861815,0.863381,0.863168,0.857049,0.857261,0.855651,0.855651,0.857261,0.857049


In [ ]:
%%time
# get some other 3 input vegetation indices  (nir, r, swir)
order = [
    # nir & rgb & swir16
    ['nir', 'red', 'swir16'],
    ['nir', 'blue', 'swir16'],
    ['nir', 'green', 'swir16'],
    # nir & rgb & swir22
    ['nir', 'red', 'swir22'],
    ['nir', 'blue', 'swir22'],
    ['nir', 'green', 'swir22'],
    # nir & rgb & swir
    ['nir', 'red', 'intensity_swir'],
    ['nir', 'blue', 'intensity_swir'],
    ['nir', 'green', 'intensity_swir'],
    # nir & rgb & rededge
    ['nir', 'red', 'intensity_rededge'],
    ['nir', 'blue', 'intensity_rededge'],
    ['nir', 'green', 'intensity_rededge'],
    # nir & rgb & coastal
    ['nir', 'red', 'coastal'],
    ['nir', 'blue', 'coastal'],
    ['nir', 'green', 'coastal'],
    
    # nir08 & rgb & swir16
    ['nir08', 'red', 'swir16'],
    ['nir08', 'blue', 'swir16'],
    ['nir08', 'green', 'swir16'],
    # nir08 & rgb & swir22
    ['nir08', 'red', 'swir22'],
    ['nir08', 'blue', 'swir22'],
    ['nir08', 'green', 'swir22'],
    # nir08 & rgb & swir
    ['nir08', 'red', 'intensity_swir'],
    ['nir08', 'blue', 'intensity_swir'],
    ['nir08', 'green', 'intensity_swir'],
    # nir08 & rgb & rededge
    ['nir08', 'red', 'intensity_rededge'],
    ['nir08', 'blue', 'intensity_rededge'],
    ['nir08', 'green', 'intensity_rededge'],
    # nir08 & rgb & coastal
    ['nir08', 'red', 'coastal'],
    ['nir08', 'blue', 'coastal'],
    ['nir08', 'green', 'coastal'],
    
    # nir09 & rgb & swir16
    ['nir09', 'red', 'swir16'],
    ['nir09', 'blue', 'swir16'],
    ['nir09', 'green', 'swir16'],
    # nir09 & rgb & swir22
    ['nir09', 'red', 'swir22'],
    ['nir09', 'blue', 'swir22'],
    ['nir09', 'green', 'swir22'],
    # nir09 & rgb & swir
    ['nir09', 'red', 'intensity_swir'],
    ['nir09', 'blue', 'intensity_swir'],
    ['nir09', 'green', 'intensity_swir'],
    # nir09 & rgb & rededge
    ['nir09', 'red', 'intensity_rededge'],
    ['nir09', 'blue', 'intensity_rededge'],
    ['nir09', 'green', 'intensity_rededge'],
    # nir09 & rgb & coastal
    ['nir09', 'red', 'coastal'],
    ['nir09', 'blue', 'coastal'],
    ['nir09', 'green', 'coastal'],
    
    # intensity_nir & rgb & swir16
    ['intensity_nir', 'red', 'swir16'],
    ['intensity_nir', 'blue', 'swir16'],
    ['intensity_nir', 'green', 'swir16'],
    # intensity_nir & rgb & swir22
    ['intensity_nir', 'red', 'swir22'],
    ['intensity_nir', 'blue', 'swir22'],
    ['intensity_nir', 'green', 'swir22'],
    # intensity_nir & rgb & swir
    ['intensity_nir', 'red', 'intensity_swir'],
    ['intensity_nir', 'blue', 'intensity_swir'],
    ['intensity_nir', 'green', 'intensity_swir'],
    # intensity_nir & rgb & rededge
    ['intensity_nir', 'red', 'intensity_rededge'],
    ['intensity_nir', 'blue', 'intensity_rededge'],
    ['intensity_nir', 'green', 'intensity_rededge'],
    # intensity_nir & rgb & coastal
    ['intensity_nir', 'red', 'coastal'],
    ['intensity_nir', 'blue', 'coastal'],
    ['intensity_nir', 'green', 'coastal'],
]

band_1, band_2, band_3 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)
band_3 = list(band_3)

funcs = [mndvi, rsr, mavi]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}_{c}' for a, b, c in order]] = func(field_features[band_1].values,
                                                                                 field_features[band_2].values,
                                                                                 field_features[band_3].values
                                                                                )

In [ ]:
field_features.head(2)

In [ ]:
%%time
# get some other 3 input vegetation indices  (nir, r, rededge1)
order = [
    # nir & rgb & rededge1
    ['nir', 'red', 'rededge1'],
    ['nir', 'blue', 'rededge1'],
    ['nir', 'green', 'rededge1'],
    ['nir', 'intensity_rgb', 'rededge1'],
    # nir & rgb & rededge2
    ['nir', 'red', 'rededge2'],
    ['nir', 'blue', 'rededge2'],
    ['nir', 'green', 'rededge2'],
    ['nir', 'intensity_rgb', 'rededge2'],
    # nir & rgb & rededge3
    ['nir', 'red', 'rededge3'],
    ['nir', 'blue', 'rededge3'],
    ['nir', 'green', 'rededge3'],
    ['nir', 'intensity_rgb', 'rededge3'],
    
    # nir08 & rgb & rededge1
    ['nir08', 'red', 'rededge1'],
    ['nir08', 'blue', 'rededge1'],
    ['nir08', 'green', 'rededge1'],
    ['nir08', 'intensity_rgb', 'rededge2'],
    # nir08 & rgb & rededge2
    ['nir08', 'red', 'rededge2'],
    ['nir08', 'blue', 'rededge2'],
    ['nir08', 'green', 'rededge2'],
    ['nir08', 'intensity_rgb', 'rededge2'],
    # nir08 & rgb & rededge3
    ['nir08', 'red', 'rededge3'],
    ['nir08', 'blue', 'rededge3'],
    ['nir08', 'green', 'rededge3'],
    ['nir08', 'intensity_rgb', 'rededge3'],
    
    # nir09 & rgb & rededge1
    ['nir09', 'red', 'rededge1'],
    ['nir09', 'blue', 'rededge1'],
    ['nir09', 'green', 'rededge1'],
    ['nir09', 'intensity_rgb', 'rededge1'],
    # nir09 & rgb & rededge2
    ['nir09', 'red', 'rededge2'],
    ['nir09', 'blue', 'rededge2'],
    ['nir09', 'green', 'rededge2'],
    ['nir09', 'intensity_rgb', 'rededge2'],
    # nir09 & rgb & rededge3
    ['nir09', 'red', 'rededge3'],
    ['nir09', 'blue', 'rededge3'],
    ['nir09', 'green', 'rededge3'],
    ['nir09', 'intensity_rgb', 'rededge3'],
    
    # intensity_nir & rgb & rededge1
    ['intensity_nir', 'red', 'rededge1'],
    ['intensity_nir', 'blue', 'rededge1'],
    ['intensity_nir', 'green', 'rededge1'],
    ['intensity_nir', 'intensity_rgb', 'rededge1'],
    # intensity_nir & rgb & rededge2
    ['intensity_nir', 'red', 'rededge2'],
    ['intensity_nir', 'blue', 'rededge2'],
    ['intensity_nir', 'green', 'rededge2'],
    ['intensity_nir', 'intensity_rgb', 'rededge2'],
    # intensity_nir & rgb & rededge3
    ['intensity_nir', 'red', 'rededge3'],
    ['intensity_nir', 'blue', 'rededge3'],
    ['intensity_nir', 'green', 'rededge3'],
    ['intensity_nir', 'intensity_rgb', 'rededge3'],
    
    # intensity or saturation based
    ['intensity_nir', 'intensity_rgb', 'intensity_rededge'],
    ['saturation_nir', 'saturation_rgb', 'saturation_rededge'],
    
]

band_1, band_2, band_3 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)
band_3 = list(band_3)

funcs = [oass, mtci]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}_{c}' for a, b, c in order]] = func(field_features[band_1].values,
                                                                                 field_features[band_2].values,
                                                                                 field_features[band_3].values
                                                                                )

In [ ]:
field_features.head(2)

In [ ]:
%%time
# get some other 3 input vegetation indices  (b, g, r)
order = [
    # rgb
    ['blue', 'green', 'red'],
    ['blue', 'red', 'green'],
    ['green', 'red', 'blue'],
    ['green', 'blue', 'red'],
    ['red', 'green', 'blue'],
    ['red', 'blue', 'green'],
    # rededge
    ['rededge1', 'rededge2', 'rededge3'],
    ['rededge1', 'rededge3', 'rededge2'],
    ['rededge2', 'rededge1', 'rededge3'],
    ['rededge2', 'rededge3', 'rededge1'],
    ['rededge3', 'rededge1', 'rededge2'],
    ['rededge3', 'rededge2', 'rededge1'],
    # nir
    ['nir', 'nir08', 'nir09'],
    ['nir', 'nir09', 'nir08'],
    ['nir08', 'nir', 'nir09'],
    ['nir08', 'nir09', 'nir'],
    ['nir09', 'nir', 'nir08'],
    ['nir09', 'nir08', 'nir'],
]

band_1, band_2, band_3 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)
band_3 = list(band_3)

funcs = [gli, rgvbi, exg, veg, cive, exgr, comi, mexg, com2]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}_{c}' for a, b, c in order]] = func(field_features[band_1].values,
                                                                                 field_features[band_2].values,
                                                                                 field_features[band_3].values
                                                                                )

In [ ]:
field_features.head(2)

In [ ]:
%%time
# get some other 3 input vegetation indices  (g, r, r_edge3)
order = [
    # rgb & rededge1
    ['green', 'red', 'rededge1'],
    ['green', 'blue', 'rededge1'],
    ['red', 'blue', 'rededge1'],
    ['red', 'green', 'rededge1'],
    ['blue', 'red', 'rededge1'],
    ['blue', 'green', 'rededge1'],
    # rgb & rededge2
    ['green', 'red', 'rededge2'],
    ['green', 'blue', 'rededge2'],
    ['red', 'blue', 'rededge2'],
    ['red', 'green', 'rededge2'],
    ['blue', 'red', 'rededge2'],
    ['blue', 'green', 'rededge2'],
    # rgb & rededge3
    ['green', 'red', 'rededge3'],
    ['green', 'blue', 'rededge3'],
    ['red', 'blue', 'rededge3'],
    ['red', 'green', 'rededge3'],
    ['blue', 'red', 'rededge3'],
    ['blue', 'green', 'rededge3'],
    # rgb & rededge
    ['green', 'red', 'intensity_rededge'],
    ['green', 'blue', 'intensity_rededge'],
    ['red', 'blue', 'intensity_rededge'],
    ['red', 'green', 'intensity_rededge'],
    ['blue', 'red', 'intensity_rededge'],
    ['blue', 'green', 'intensity_rededge'],
    # rgb & nir
    ['green', 'red', 'intensity_nir'],
    ['green', 'blue', 'intensity_nir'],
    ['red', 'blue', 'intensity_nir'],
    ['red', 'green', 'intensity_nir'],
    ['blue', 'red', 'intensity_nir'],
    ['blue', 'green', 'intensity_nir'],
    # rgb & swir
    ['green', 'red', 'intensity_swir'],
    ['green', 'blue', 'intensity_swir'],
    ['red', 'blue', 'intensity_swir'],
    ['red', 'green', 'intensity_swir'],
    ['blue', 'red', 'intensity_swir'],
    ['blue', 'green', 'intensity_swir'],
    # rgb & coastal
    ['green', 'red', 'coastal'],
    ['green', 'blue', 'coastal'],
    ['red', 'blue', 'coastal'],
    ['red', 'green', 'coastal'],
    ['blue', 'red', 'coastal'],
    ['blue', 'green', 'coastal'],
]

band_1, band_2, band_3 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)
band_3 = list(band_3)

funcs = [mcari, tcari]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}_{c}' for a, b, c in order]] = func(field_features[band_1].values,
                                                                                 field_features[band_2].values,
                                                                                 field_features[band_3].values
                                                                                )

In [ ]:
field_features.head(2)

In [ ]:
%%time
# get some other 3 input vegetation indices  (r_edge1, r_edge2, coastal)
order = [
    # rededge & coastal
    ['rededge1', 'rededge2', 'coastal'],
    ['rededge1', 'rededge3', 'coastal'],
    ['rededge2', 'rededge3', 'coastal'],
    ['rededge2', 'rededge1', 'coastal'],
    ['rededge3', 'rededge1', 'coastal'],
    ['rededge3', 'rededge2', 'coastal'],
    # rededge & nir
    ['rededge1', 'rededge2', 'intensity_nir'],
    ['rededge1', 'rededge3', 'intensity_nir'],
    ['rededge2', 'rededge3', 'intensity_nir'],
    ['rededge2', 'rededge1', 'intensity_nir'],
    ['rededge3', 'rededge1', 'intensity_nir'],
    ['rededge3', 'rededge2', 'intensity_nir'],
    # rededge & rgb
    ['rededge1', 'rededge2', 'intensity_rgb'],
    ['rededge1', 'rededge3', 'intensity_rgb'],
    ['rededge2', 'rededge3', 'intensity_rgb'],
    ['rededge2', 'rededge1', 'intensity_rgb'],
    ['rededge3', 'rededge1', 'intensity_rgb'],
    ['rededge3', 'rededge2', 'intensity_rgb'],
    # rededge & swir
    ['rededge1', 'rededge2', 'intensity_swir'],
    ['rededge1', 'rededge3', 'intensity_swir'],
    ['rededge2', 'rededge3', 'intensity_swir'],
    ['rededge2', 'rededge1', 'intensity_swir'],
    ['rededge3', 'rededge1', 'intensity_swir'],
    ['rededge3', 'rededge2', 'intensity_swir'],
]

band_1, band_2, band_3 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)
band_3 = list(band_3)

funcs = [mresr]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}_{c}' for a, b, c in order]] = func(field_features[band_1].values,
                                                                                 field_features[band_2].values,
                                                                                 field_features[band_3].values
                                                                                )


In [ ]:
field_features.head(2)

In [ ]:
%%time
# get some other 4 input vegetation indices  (r, r_edge1, r_edge2, r_edge3)
order = [
    # rgb & rededge
    ['red', 'rededge1', 'rededge2', 'rededge3'],
    ['blue', 'rededge1', 'rededge2', 'rededge3'],
    ['green', 'rededge1', 'rededge2', 'rededge3'],
    ['intensity_rgb', 'rededge1', 'rededge2', 'rededge3'],
    # coastal & rededge
    ['coastal', 'rededge1', 'rededge2', 'rededge3'],
    ['coastal', 'rededge1', 'rededge2', 'rededge3'],
    ['coastal', 'rededge1', 'rededge2', 'rededge3'],
    # nir & rededge
    ['nir', 'rededge1', 'rededge2', 'rededge3'],
    ['nir08', 'rededge1', 'rededge2', 'rededge3'],
    ['nir09', 'rededge1', 'rededge2', 'rededge3'],
    ['intensity_nir', 'rededge1', 'rededge2', 'rededge3'],
    # swir & rededge
    ['swir16', 'rededge1', 'rededge2', 'rededge3'],
    ['swir22', 'rededge1', 'rededge2', 'rededge3'],
    ['intensity_swir', 'rededge1', 'rededge2', 'rededge3'],
]


band_1, band_2, band_3, band_4 = zip(*order)
band_1 = list(band_1)
band_2 = list(band_2)
band_3 = list(band_3)
band_4 = list(band_4)

funcs = [s2rep, brightness, greenness, yellowness, nonesuch]
for func in funcs:
    field_features[[f'{func.__name__}_{a}_{b}_{c}_{d}' for a, b, c, d in order]] = func(field_features[band_1].values,
                                                                                 field_features[band_2].values,
                                                                                 field_features[band_3].values,
                                                                                 field_features[band_4].values
                                                                                )


In [ ]:
field_features.head(2)

In [ ]:
field_features.dtypes.unique()

In [ ]:
field_features.columns[field_features.dtypes == 'int']

In [ ]:
df = field_features.drop(columns=['field_id'])
drop_cols = df.columns[np.isinf(df).any()].tolist()

In [ ]:
drop_cols2 = df.columns[df.isnull().sum() != 0].tolist()

In [ ]:
len(drop_cols), len(drop_cols2)

In [ ]:
drop_cols = list(set(drop_cols).union(set(drop_cols2)))

In [ ]:
len(drop_cols)

In [ ]:
field_features.drop(columns=drop_cols, inplace=True)

In [ ]:
field_features.shape

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%%time
agg_columns = [col for col in field_features.columns if col not in ['field_id', 'Train']]

funcs = {
#     'mean': np.mean,
    'median': np.median,
    'std': np.std,
    'min': np.min,
    'max': np.max,
    'kurtosis':  lambda x: pd.DataFrame(x).kurtosis().fillna(0),
    'skew': lambda x: pd.DataFrame(x).skew().fillna(0), 
}
field_features_all = field_features[['field_id', *agg_columns]].groupby('field_id')[agg_columns].mean().rename(columns={name: f'{name}_mean' for name in agg_columns}).reset_index()
for key, value in tqdm(funcs.items()):
    feature_names = {name: f'{name}_{key}' for name in agg_columns}
    df = field_features[['field_id', *agg_columns]].groupby('field_id')[agg_columns].parallel_apply(value)
    df = df.rename(columns=feature_names).reset_index() 
    field_features_all = field_features_all.merge(df, on='field_id', how='left')
   
field_features_all.shape

In [ ]:
field_features_all.head(2)

In [ ]:
train_code = field_features.groupby('field_id')[['field_id', 'Train']].head(1) 

field_features_all = field_features_all.merge(train_code, on='field_id', how='left')

In [ ]:
field_features_all.head(2)

#  Remove the nan cols

In [ ]:
df_nan = (field_features_all.isnull().sum()[field_features_all.isnull().sum() != 0] / field_features_all.shape[0])
nan_columns = df_nan[df_nan > .4].index.tolist()

In [ ]:
field_features_all.drop(columns=nan_columns, inplace=True)

In [ ]:
field_features_all.shape

In [ ]:
field_features_all.shape

In [ ]:
train_field_crop.groupby('field_id').head(1).shape

In [ ]:
train_df = field_features_all[field_features_all.Train == 1].drop(columns=['Train'])
test_df = field_features_all[field_features_all.Train == 0].drop(columns=['Train'])

train_df = train_df.merge(train_field_crop.groupby('field_id').head(1)[['field_id', 'crop']],
                          on='field_id', how='left'
                         )
train_df.shape, test_df.shape

In [ ]:
drop_columns = []

In [ ]:
%%time

num_feature = [col for col in train_df.columns if col not in ['field_id', 'crop']]

# compute the correlation
corr = train_df[num_feature].corr()
corr_values = np.triu(corr.values,  k=1)  # turn anything below leading diagonal 0 (+ diagonal)

# compute the mask
pos_mask = corr_values >= 0.999
neg_mask = corr_values <= -0.995
mask = pos_mask | neg_mask
columns_mask = np.any(mask, axis=0)

# get the extracted columns
drop_columns = corr.columns[columns_mask].tolist()

print(f'{len(drop_columns)} columns to be dropped\n')
if len(drop_columns) < 20:
    print(drop_columns)

In [ ]:
train_df.drop(drop_columns,inplace=True,axis =1 )
test_df.drop(drop_columns,inplace=True,axis =1 )

**Drop constant features if available:**

In [ ]:
columns_to_drop = []

In [ ]:
num_feature = [col for col in train_df.columns if col not in ['field_id', 'crop']]

std_df = train_df[num_feature].std()
columns_to_drop = std_df[std_df == 0].index.tolist()

print(f'{len(columns_to_drop)} columns to be dropped\n')
if len(columns_to_drop) < 20:
    print(columns_to_drop)

In [ ]:
train_df.drop(columns=columns_to_drop,inplace=True)
test_df.drop(columns=columns_to_drop,inplace=True)

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.head(3)

In [ ]:
cols = list(train_df.columns)
median_cols = [x for x in cols if 'median' in x]
median_cols

In [ ]:
len(median_cols)

In [ ]:
%%time
# save features
os.makedirs('data', exist_ok=True)
train_df.to_csv('data/train_new2_features.csv', index=False)
test_df.to_csv('data/test_new2_features.csv', index=False)